In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd
predict_df = pd.read_csv('/content/drive/MyDrive/IEOR 4212 Project/Datasets/data_to_map.csv')
f = open("/content/drive/MyDrive/IEOR 4212 Project/Datasets/new_england_geojson.geojson", 'r')
content = f.read()

In [3]:
predict_df['zip_code'] = predict_df['zip_code'].apply(lambda x: str(x).zfill(5))

In [4]:
predict_df.groupby(['Region','Year'])['proportion_load'].sum()

Region  Year
CT      2025    12.0
        2026    12.0
        2027    12.0
        2028    12.0
        2029    12.0
                ... 
WCMA    2041    12.0
        2042    12.0
        2043    12.0
        2044    12.0
        2045    12.0
Name: proportion_load, Length: 168, dtype: float64

In [ ]:
content

Output hidden; open in https://colab.research.google.com to view.

##Timeseries Choropleth Map for EV Load in Each Zipcode

In [5]:
# Convert from json to a python object (a dictionary in this case)
import json
import folium

geojson_data = json.loads(content)
m=folium.Map(location = [44,-71],zoom_start=6)

In [ ]:
geojson_data

Output hidden; open in https://colab.research.google.com to view.

##ATTEMPT 2

In [7]:
# Create quantile bins for color
import numpy as np

data_to_map = predict_df[['zip_code', 'timestamp', 'ev_load']].copy()
data_to_map['ev_load'] = data_to_map['ev_load'] * 1000000 # convert from megawatts to watts
data_to_map['timestamp'] = pd.to_datetime(data_to_map['timestamp'])
data_to_map['time_index'] = (data_to_map['timestamp'].astype(np.int64) / 10**9).astype(int)

quantile_bins = data_to_map['ev_load'].quantile([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]).tolist()

def get_color(value):
    if value < quantile_bins[1]: return '#006837'
    elif value < quantile_bins[2]: return '#1a9850'
    elif value < quantile_bins[3]: return '#66bd63'
    elif value < quantile_bins[4]: return '#a6d96a'
    elif value < quantile_bins[5]: return '#d9ef8b'
    elif value < quantile_bins[6]: return '#ffffbf'
    elif value < quantile_bins[7]: return '#fee08b'
    elif value < quantile_bins[8]: return '#fdae61'
    elif value < quantile_bins[9]: return '#f46d43'
    elif value < quantile_bins[10]: return '#d73027'
    else: return '#a50026'

bins = quantile_bins

In [10]:
import folium
from folium.plugins import TimeSliderChoropleth
import numpy as np
import pandas as pd

m = folium.Map(location=[44, -71], zoom_start=7, tiles='OpenStreetMap')

styledict = {}
for idx, row in data_to_map.iterrows():
    zipcode_str = str(int(row['zip_code'])).zfill(5)
    time_idx = str(row['time_index'])

    if zipcode_str not in styledict:
        styledict[zipcode_str] = {}

    styledict[zipcode_str][time_idx] = {
        'color': get_color(row['ev_load']),
        'opacity': 0.8
    }

features_to_include = []
for feature in geojson_data['features']:
    zip_code = feature['properties'].get('ZCTA5CE10')
    if zip_code in styledict:
        feature['id'] = zip_code
        features_to_include.append(feature)

filtered_geojson = {
    'type': 'FeatureCollection',
    'features': features_to_include
}

# Add timeslider
TimeSliderChoropleth(
    filtered_geojson,
    styledict=styledict
).add_to(m)

# used ai to generate legend
legend_html = f"""
<div style="position: fixed;
    bottom: 50px; right: 50px; width: 200px; height: 380px;
    background-color: white; border:2px solid grey; z-index:9999;
    font-size:13px; padding: 10px">
    <p style="margin-bottom:10px; font-weight:bold;">EV Load (Watts)</p>
    <p><i style="background:#006837; width:20px; height:20px; float:left; margin-right:5px; display:block;"></i> < {bins[1]:.2f}</p>
    <p><i style="background:#1a9850; width:20px; height:20px; float:left; margin-right:5px; display:block;"></i> {bins[1]:.2f} - {bins[2]:.2f}</p>
    <p><i style="background:#66bd63; width:20px; height:20px; float:left; margin-right:5px; display:block;"></i> {bins[2]:.2f} - {bins[3]:.2f}</p>
    <p><i style="background:#a6d96a; width:20px; height:20px; float:left; margin-right:5px; display:block;"></i> {bins[3]:.2f} - {bins[4]:.2f}</p>
    <p><i style="background:#d9ef8b; width:20px; height:20px; float:left; margin-right:5px; display:block;"></i> {bins[4]:.2f} - {bins[5]:.2f}</p>
    <p><i style="background:#ffffbf; width:20px; height:20px; float:left; margin-right:5px; display:block;"></i> {bins[5]:.2f} - {bins[6]:.2f}</p>
    <p><i style="background:#fee08b; width:20px; height:20px; float:left; margin-right:5px; display:block;"></i> {bins[6]:.2f} - {bins[7]:.2f}</p>
    <p><i style="background:#fdae61; width:20px; height:20px; float:left; margin-right:5px; display:block;"></i> {bins[7]:.2f} - {bins[8]:.2f}</p>
    <p><i style="background:#f46d43; width:20px; height:20px; float:left; margin-right:5px; display:block;"></i> {bins[8]:.2f} - {bins[9]:.2f}</p>
    <p><i style="background:#d73027; width:20px; height:20px; float:left; margin-right:5px; display:block;"></i> {bins[9]:.2f} - {bins[10]:.2f}</p>
    <p><i style="background:#a50026; width:20px; height:20px; float:left; margin-right:5px; display:block;"></i> > {bins[10]:.2f}</p>
</div>
"""

m.get_root().html.add_child(folium.Element(legend_html))

In [15]:
m.save("New England EV Load 2025-2045.html")

In [14]:
import folium
from folium.plugins import TimeSliderChoropleth
import numpy as np
import pandas as pd

m = folium.Map(location=[44, -71], zoom_start=7, tiles='OpenStreetMap')

styledict = {}
for idx, row in data_to_map.iterrows():
    zipcode_str = str(int(row['zip_code'])).zfill(5)
    time_idx = str(row['time_index'])

    if zipcode_str not in styledict:
        styledict[zipcode_str] = {}

    styledict[zipcode_str][time_idx] = {
        'color': get_color(row['ev_load']),
        'opacity': 0.8
    }

features_to_include = []
for feature in geojson_data['features']:
    zip_code = feature['properties'].get('ZCTA5CE10')
    if zip_code in styledict:
        feature['id'] = zip_code
        features_to_include.append(feature)

filtered_geojson = {
    'type': 'FeatureCollection',
    'features': features_to_include
}

# Add timeslider
TimeSliderChoropleth(
    filtered_geojson,
    styledict=styledict
).add_to(m)


# Used AI to customize html

# Add title
title_html = """
<h3 style="position: fixed; top: 10px; left: 50%; transform: translateX(-50%);
           z-index: 9999; background-color: white; padding: 5px 15px; margin: 0;">
    New England Monthly EV Load Forecast 2025-2045
</h3>
"""

m.get_root().html.add_child(folium.Element(title_html))

# Keyboard controls and slider styling
slider_css = """
<style>
    .leaflet-bottom.leaflet-left {
        width: 90% !important;
        left: 5% !important;
        bottom: 20px !important;
    }

    input[type="range"].time-slider {
        width: 100% !important;
        height: 15px !important;
    }
</style>

<script>
document.addEventListener('DOMContentLoaded', function() {
    setTimeout(function() {
        var slider = document.querySelector('input[type="range"].time-slider');
        if (slider) {
            // Add keyboard navigation
            document.addEventListener('keydown', function(e) {
                if (e.key === 'ArrowRight' || e.key === '>') {
                    e.preventDefault();
                    slider.stepUp();
                    slider.dispatchEvent(new Event('input', { bubbles: true }));
                } else if (e.key === 'ArrowLeft' || e.key === '<') {
                    e.preventDefault();
                    slider.stepDown();
                    slider.dispatchEvent(new Event('input', { bubbles: true }));
                }
            });
        }
    }, 1000);
});
</script>
"""

m.get_root().html.add_child(folium.Element(slider_css))

# Legend
legend_html = f"""
<div style="position: fixed;
    bottom: 100px; right: 20px; width: 180px;
    background-color: white; border:2px solid grey; z-index:9999;
    font-size:12px; padding: 8px">
    <p style="margin-bottom:8px; font-weight:bold;">EV Load (Watts)</p>
    <p><i style="background:#006837; width:18px; height:18px; float:left; margin-right:5px; display:block;"></i> < {bins[1]:.2f}</p>
    <p><i style="background:#1a9850; width:18px; height:18px; float:left; margin-right:5px; display:block;"></i> {bins[1]:.2f} - {bins[2]:.2f}</p>
    <p><i style="background:#66bd63; width:18px; height:18px; float:left; margin-right:5px; display:block;"></i> {bins[2]:.2f} - {bins[3]:.2f}</p>
    <p><i style="background:#a6d96a; width:18px; height:18px; float:left; margin-right:5px; display:block;"></i> {bins[3]:.2f} - {bins[4]:.2f}</p>
    <p><i style="background:#d9ef8b; width:18px; height:18px; float:left; margin-right:5px; display:block;"></i> {bins[4]:.2f} - {bins[5]:.2f}</p>
    <p><i style="background:#ffffbf; width:18px; height:18px; float:left; margin-right:5px; display:block;"></i> {bins[5]:.2f} - {bins[6]:.2f}</p>
    <p><i style="background:#fee08b; width:18px; height:18px; float:left; margin-right:5px; display:block;"></i> {bins[6]:.2f} - {bins[7]:.2f}</p>
    <p><i style="background:#fdae61; width:18px; height:18px; float:left; margin-right:5px; display:block;"></i> {bins[7]:.2f} - {bins[8]:.2f}</p>
    <p><i style="background:#f46d43; width:18px; height:18px; float:left; margin-right:5px; display:block;"></i> {bins[8]:.2f} - {bins[9]:.2f}</p>
    <p><i style="background:#d73027; width:18px; height:18px; float:left; margin-right:5px; display:block;"></i> {bins[9]:.2f} - {bins[10]:.2f}</p>
    <p><i style="background:#a50026; width:18px; height:18px; float:left; margin-right:5px; display:block;"></i> > {bins[10]:.2f}</p>
</div>
"""

m.get_root().html.add_child(folium.Element(legend_html))

m


Output hidden; open in https://colab.research.google.com to view.